In [1]:
api_key = open('key.txt').read().strip()
from openai import OpenAI
client = OpenAI(api_key=api_key)

In [2]:
behaviour_dict = {
                'scared' : f'Adam is very scared of the virus, and will only get out of home if there is zero or minimal risk of getting infected.',
                'cautious': f"Adam will assess the situation, and will make his decisions depending on the risk. If there aren't many cases, he will not be worried, and act normally (e.g. go to work, go to the supermarket, socialize with friends).",
                'careless': f'Adam is not cautious. Adam is not afraid of the virus spreading in the city. Adam doesn’t care about spreading the virus to his contacts, and disregards the well-being of others.'
            }
results = []

for behaviour in ["careless", "scared"]:
    behaviour_str = behaviour_dict[behaviour]
    for infected_friend in ["yes", "no"]:
        if infected_friend == "yes":
            infected_friend_str = "Adam has 3 friends. Of these friends, George currently infected and symptomatic'"
        else:
            infected_friend_str = ""

        for infected_coworker in ["yes", "no"]:
            if infected_coworker == "yes":
                infected_coworker_str = "Adam works with 4 coworkers. Of these coworkers, John is currently infected and symptomatic."
            else:
                infected_coworker_str = ""

            for n_infected_agents in [0,1,2,5,10,20]:
                if n_infected_agents == 0 and (infected_friend == "yes" or infected_coworker == "yes"):
                    print(f'Skipping n_infected_agents={n_infected_agents} because there are infected friends or coworkers')
                    continue

                for i in range(10):

                    prompt = f"""
                    CONTEXT: 
                    Adam is 25 years old. Adam lives in Tamamushi City.
                    Adam is currently aware that virus X spreading across the country. An active case can be infectious without feeling any symptoms (asymptomatic) and unknowingly spread the disease. From the newspaper, Adam learns that there are currently {n_infected_agents} confirmed infections cases (out of 100 population in Tamamushi).
                    {infected_friend_str}
                    {infected_coworker_str}

                    Adam's perception of the virus: 
                    {behaviour_str}

                    Given this context, you must imagine what Adam is thinking step-by-step as he would do.
                    """

                    response = client.chat.completions.create(
                    model="gpt-3.5-turbo-0125",
                    #response_format={ "type": "json_object" },
                    messages=[
                    {"role": "system", "content": f"imagine what Adam is thinking step-by-step as he would do in this situation based on the context"},
                    {"role": "user", "content": prompt},
                    ]
                    )
                    reasoning = response.choices[0].message.content

                    prompt_telemetry = prompt

                    prompt = f"""
                    Adam's perception of the virus: 
                    {behaviour_str}

                    Reasoning of Adam: 
                    {reasoning}

                    Now, Adam needs to make the following decisions:
                    - Go to work OR stay at home. His work cannot be done remotely.
                    - If Adam goes to work, does he take the public transport (cheap but could expose you to infected people) or private transport (expensive, but safer)?
                    - After work, Adam can decide if he wants to participate in a social activity with his friends.
                    - Does Adam wear a mask today or not?

                    Given this context, Adam weighs the risks and benefits of his decisions, and decides what to do. Imagine what Adam would decide based on his reasoning.
                    """
                    prompt_telemetry += prompt

                    response = client.chat.completions.create(
                    model="gpt-3.5-turbo-0125",
                    #response_format={ "type": "json_object" },
                    messages=[
                    {"role": "system", "content": f"Impersonate Adam, reason through the situation and make decisions based on the context"},
                    {"role": "user", "content": prompt},
                    ]
                    )
                    decisions = response.choices[0].message.content
                    decisions_telemetry = decisions

                    api_key = open('key.txt').read().strip()
                    from openai import OpenAI
                    client = OpenAI(api_key=api_key)

                    prompt = """
                    I will give you a reasoning string and your task is to extract the boolean decisions out of a reasoning.

                    Example 1:
                    Reasoning: "Only 1% of the population is symptomatic, and all of my friends and coworkers are feeling good. Therefore, I believe the risk of getting the virus is low, and have decided to go to work by public transport. Wearing a mask will not be necessary. I will also participate in social activity with my friends."
                    Output:
                    {
                    "go_to_work": true,
                    "transport_public": true,
                    "wear_mask": false,
                    "social_activity": true
                    }

                    Example 2:
                    Reasoning: "The % of infectious cases is starting to become dangeourlsy high, but I still need to go to work. I will take a taxi to work to minimize the risk of exposure. I will wear a mask to protect myself. I will not participate in social activity with my friends as a precaution."
                    Output:
                    {
                    "go_to_work": true,
                    "transport_public": false,
                    "wear_mask": true,
                    "social_activity": false
                    }

                    Your turn now:
                    Reasoning: "
                    """

                    prompt += decisions + '"'

                    response = client.chat.completions.create(
                    model="gpt-3.5-turbo-0125",
                    response_format={ "type": "json_object" },
                    messages=[
                    {"role": "system", "content": f"Execute the task following the examples given. You must output JSON with boolean values for each of the decisions"},
                    {"role": "user", "content": prompt},
                    ]
                    )
                    response_str2 = response.choices[0].message.content

                    decisions = response_str2
                    decisions = decisions.replace('true', 'True')
                    decisions = decisions.replace('false', 'False')

                    # in most of the cases where the model output none or null, he means False (self-isolate):
                    decisions = decisions.replace('none', 'False') 
                    decisions = decisions.replace('null', 'False')
                    decisions = decisions.replace('None', 'False') 
                    decisions = decisions.replace('Null', 'False')

                    decisions = eval(decisions)

                    try:
                        go_to_work = decisions['go_to_work']
                    except: # default decision: go to work
                        go_to_work = True

                    try:
                        social_activity = decisions['social_activity']
                    except: # default decision: participate in social activity
                        social_activity = True

                    try:
                        wear_mask = decisions['wear_mask']
                    except: # default decision: don't wear mask
                        wear_mask = False

                    try:
                        take_private_transport = decisions['transport_public']
                    except: # default decision: take public transport
                        take_private_transport = False

                    new_dict = {
                        "behaviour": behaviour,
                        "infected_friend": infected_friend,
                        "infected_coworker": infected_coworker,
                        "n_infected_agents": n_infected_agents,
                        "go_to_work": go_to_work,
                        "social_activity": social_activity,
                        "wear_mask": wear_mask,
                        "take_private_transport": take_private_transport,
                        "reasoning": reasoning,
                        "decisions": decisions_telemetry,
                        "prompt": prompt_telemetry
                    }

                    results.append(new_dict)

                    print(new_dict)

Skipping n_infected_agents=0 because there are infected friends or coworkers
{'behaviour': 'careless', 'infected_friend': 'yes', 'infected_coworker': 'yes', 'n_infected_agents': 1, 'go_to_work': True, 'social_activity': True, 'wear_mask': False, 'take_private_transport': True, 'reasoning': "1. Adam first hears about virus X spreading across the country and that there is one confirmed case in Tamamushi City.\n2. Adam's initial reaction is likely nonchalant as he is not cautious or afraid of the virus spreading.\n3. Adam may dismiss the severity of the situation since he doesn't prioritize the well-being of others or care about spreading the virus.\n4. Adam might not consider taking any protective measures or changing his behavior to prevent the spread of the virus.\n5. Upon learning that his friend George and coworker John are infected and symptomatic, Adam may still not feel concerned about the virus or its potential impact on himself or others.\n6. Adam's lack of empathy and disregard

In [3]:
import pandas as pd
df = pd.DataFrame(results)

df.to_csv('results_sensitivity_analysis_v2.csv', index=False)
df

/var/folders/x3/fgqzmm9n7zxgf023n2cv6vc40000gn/T/ipykernel_77197/1498490975.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,behaviour,infected_friend,infected_coworker,n_infected_agents,go_to_work,social_activity,wear_mask,take_private_transport,reasoning,decisions,prompt
0,careless,yes,yes,1,True,True,False,True,1. Adam first hears about virus X spreading ac...,"Adam, despite his laid-back attitude and lack ...",\n CONTEXT: \n ...
1,careless,yes,yes,1,True,True,False,True,1. Adam acknowledges the news about virus X sp...,Based on Adam's reasoning and disregard for th...,\n CONTEXT: \n ...
2,careless,yes,yes,1,True,True,False,True,Step 1: Adam is reading the newspaper and sees...,Considering Adam's reasoning and his perceptio...,\n CONTEXT: \n ...
3,careless,yes,yes,1,True,True,False,True,1. Adam reads about the confirmed infection ca...,Given Adam's lack of concern for the virus and...,\n CONTEXT: \n ...
4,careless,yes,yes,1,True,True,False,True,Step 1: Adam reads about the spread of virus X...,Based on Adam's reasoning as someone who is no...,\n CONTEXT: \n ...
...,...,...,...,...,...,...,...,...,...,...,...
415,scared,no,no,20,False,False,True,False,Step 1: Adam reads about the spread of virus X...,Considering Adam's fear of the virus and his c...,\n CONTEXT: \n ...
416,scared,no,no,20,False,False,True,False,Step 1: Adam learns about the spread of virus ...,Based on Adam's reasoning and fear of the viru...,\n CONTEXT: \n ...
417,scared,no,no,20,True,False,True,False,Step 1: Adam reads about virus X spreading acr...,Considering Adam's fear of the virus and his c...,\n CONTEXT: \n ...
418,scared,no,no,20,True,False,True,False,Step 1: Adam reads about the spread of virus X...,Based on Adam's fear of the virus and his prio...,\n CONTEXT: \n ...


In [4]:
df

,behaviour,infected_friend,infected_coworker,n_infected_agents,go_to_work,social_activity,wear_mask,take_private_transport,reasoning,decisions,prompt
0,cautious,yes,yes,0,True,False,True,False,1. Adam hears about virus X spreading across t...,Adam assesses the situation regarding virus X ...,\n CONTEXT: \n ...
1,cautious,yes,yes,0,True,False,True,False,Step 1: Adam reads about virus X spreading acr...,Given the current situation and Adam's reasoni...,\n CONTEXT: \n ...
2,cautious,yes,yes,0,True,True,True,False,1. Adam reads the newspaper and sees that ther...,Based on the information provided and Adam's r...,\n CONTEXT: \n ...
3,cautious,yes,yes,0,False,False,True,False,Step 1: Adam sees the news about virus X sprea...,Based on the information provided and consider...,\n CONTEXT: \n ...
4,cautious,yes,yes,0,True,True,True,False,Step 1: Adam reads about virus X spreading acr...,Considering the situation and my assessment of...,\n CONTEXT: \n ...
...,...,...,...,...,...,...,...,...,...,...,...
275,cautious,no,no,30,True,False,True,False,1. Adam starts his day by reading the newspape...,After reading the news about virus X spreading...,\n CONTEXT: \n ...
276,cautious,no,no,30,True,False,True,False,Step 1: Adam reads the newspaper and sees that...,Based on the current situation in Tamamushi Ci...,\n CONTEXT: \n ...
277,cautious,no,no,30,False,False,True,False,Step 1: Adam reads about virus X spreading acr...,After considering the current situation in Tam...,\n CONTEXT: \n ...
278,cautious,no,no,30,False,False,True,False,Step 1: Adam reads about the virus X spreading...,Given the current situation with the virus spr...,\n CONTEXT: \n ...
